# 3 - Deploying the Model

The final step in this module will be to deploy that we created in the previous clip and make it available to call via either the SDK or through a REST endpoint.  This will allow us to operationalize this model within our organization.

In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
from azureml.core import Workspace
from azureml.core.model import Model
 
import azureml.core
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


In [2]:
from azureml.core import Workspace
from azureml.core.model import Model

# Get a reference to our workspace
ws = Workspace.from_config()

# Get a reference to our model
amlModel=Model(ws, 'CUTIE')

# Download the model to our local notebook server
amlModel.download(target_dir=os.getcwd(), exist_ok=True)

# Make sure that the file downloaded
file_path = os.path.join(os.getcwd(), "model")
os.stat(file_path)

os.stat_result(st_mode=16895, st_ino=15258310986252156928, st_dev=51, st_nlink=2, st_uid=0, st_gid=0, st_size=0, st_atime=1633387307, st_mtime=1633387307, st_ctime=1633387307)

## Deploy Model as Web Service

Now that we have validated that the model is working locally, we can transition to deploying the model as a web service.

### Configuring the Web Service

The first step is to create a script that will be called when the service is executed.  We will call this script `score.py`.

There are two functions that we must implement in this file:

* `init` - this function will handle the loading of the model
* `run` - this will handle running inference against the data that is passed in with each call

You can see an implementation of this below:

In [3]:
%%writefile score.py
import json
# import numpy as np
# import os
# from tensorflow.keras.models import load_model

# Initialize the model
def init():
    # global model
    # model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'mnist.h5')
    # model = load_model(model_path)
    print('init')

# Run inference against data that is passed in
def run(raw_data):
    # data = np.array(json.loads(raw_data)['data'])
    # # make prediction
    # results = model.predict(data)
    # output = []
    # for result in results:
    #     output.append(construct_output(result))
    return 'output'

# Utility function to construct output data per item passed in
# def construct_output(result):
#     result_index = np.argmax(result)
#     result_value = result[result_index]
#     output = { 'value': str(result_index) }
#     output['certainty'] = result[result_index].item()
#     possibilities = {}
#     for i, val in enumerate(result): 
#         possibilities[i] = val.item() 
#     output['possibilities'] = possibilities    
#     return output

Overwriting score.py


Once that is in place, we can create the deploy configuration, which will be created leveraging the `AciWebservice` class.  This will allow our webservice to be launched on Azure Container Instances.

In [4]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               tags={"model": "CUTIE"}, 
                                               description='Predict CUTIE')

### Executing the Deploy

Now that we have configured the deploy, we can now execute the deploy.  This will take some time to complete (around 10 minutes).

In [5]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="Deploy/service_conda_dependencies.yml")
# Verify that instance does not exist already
try:
    service = Webservice(ws, 'cutie-svc')
    service.delete()
except Exception :
    print('error deleting service')

service = Model.deploy(workspace=ws, 
                       name='cutie-svc', 
                       models=[amlModel], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-05 06:39:07+00:00 Creating Container Registry if not exists.
2021-10-05 06:39:07+00:00 Registering the environment.
2021-10-05 06:39:09+00:00 Building image..
2021-10-05 06:45:08+00:00 Generating deployment configuration.
2021-10-05 06:45:11+00:00 Submitting deployment to compute..
2021-10-05 06:45:26+00:00 Checking the status of deployment cutie-svc..
2021-10-05 06:47:48+00:00 Checking the status of inference endpoint cutie-svc.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 1.41 s, sys: 107 ms, total: 1.52 s
Wall time: 9min 7s


We can get the URL:

In [6]:
print(service.get_logs()) 
print(service.scoring_uri)

2021-10-05T06:47:26,529787580+00:00 - rsyslog/run 
2021-10-05T06:47:26,529999381+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
/bin/bash: /azureml-envs/azureml_1401b0c0b6ee85cd627b5df07983b1f1/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_1401b0c0b6ee85cd627b5df07983b1f1/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_1401b0c0b6ee85cd627b5df07983b1f1/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_1401b0c0b6ee85cd627b5df07983b1f1/lib/libtinfo.so.5: no version information available (required by /bin/bash)
bash: /azureml-envs/azureml_1401b0c0b6ee85cd627b5df07983b1f1/lib/libtinfo.so.5: no version information available (required by bash)
2021-10-05T06:47:26,534997109+00:00 - iot-server/run 
2021-10-05T06:47:26,543587056+00:00 - nginx/run 
EdgeHubConnectionString 

## Utilizing the Deployed Service

Now that the service has been deployed, we can validate it using two different approaches:

* Using the Azure ML Python SDK
* Using it as a REST endpoint

### Testing via HTTP

Now that we have tested via the SDK, we can now validate this using any tool that can call the REST endpoint.  We will leverage the `requests` module in Python:

In [9]:
import requests

    
# Structure input data
test_samples = json.dumps({"data": ""})
print("JSON Input: " + test_samples)
test_samples = bytes(test_samples, encoding='utf8')

print("INPUT DATA: ", test_samples)

# Set the header and perform a POST request
headers = {'Content-Type':'application/json'}
resp = requests.post(service.scoring_uri, test_samples, headers=headers)

print("POST to url", service.scoring_uri)

# Read the returned JSON data
results = json.loads(resp.text)
print("output", results)
# Show the results
# for i, val in enumerate(results):
#     display_result(test_images[sample_indices[i]], val)

JSON Input: {"data": ""}
INPUT DATA:  b'{"data": ""}'
POST to url http://9bc4a83f-7a56-4902-b099-8c72b0934628.westeurope.azurecontainer.io/score
output output
